# Hyperparamètre

In [1]:
train_keras = True
train_qkeras =True
model_name = 'AlexNet_Weights/AlexNet_Cifar10_Keras.h5'
qmodel_name_8 = 'AlexNet_Weights/AlexNet_Cifar10_QKeras_8.h5'
qmodel_name_16 = 'AlexNet_Weights/AlexNet_Cifar10_QKeras_16.h5'
epochs = 200
batch_size = 512
lr = 0.001
log_dir_keras = 'AlexNet_Cifar10_LOG/Keras'
log_dir_qkeras_8 = 'AlexNet_Cifar10_LOG/QKeras_8'
log_dir_qkeras_16 = 'AlexNet_Cifar10_LOG/QKeras_16'

# Dataset Part

In [2]:
import tensorflow as tf
import keras
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

2023-06-28 15:19:13.551843: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
devices = tf.config.list_physical_devices()

for device in devices:
    print(device)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2023-06-28 15:19:16.035478: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-28 15:19:16.086386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 15:19:16.086748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-28 15:19:16.086785: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-28 15:19:16.092713: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-28 15:19:16.092811: I tensorflow/stream_executor

In [4]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Keras Part

In [5]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Model

filters_per_conv_layer = [64, 4, 8, 16, 16]
neurons_per_dense_layer = [16, 10]

x = x_in = Input((32, 32, 3)) 

n_classes = 10 

for i, f in enumerate(filters_per_conv_layer):
    print(('Adding convolutional block {} with N={} filters').format(i, f))
    x = Conv2D(
        int(f),
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.001),
        use_bias=False,
        name='conv_{}'.format(i),
    )(x)
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu', name='conv_act_%i' % i)(x)
    if i == 0 or i == 1 or i == 4:  # Apply pooling after layers 1, 2 and 5
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool_{}'.format(i))(x)
x = Flatten()(x)

for i, n in enumerate(neurons_per_dense_layer):
    print(('Adding dense block {} with N={} neurons').format(i, n))
    x = Dense(n, kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.001), name='dense_%i' % i, use_bias=False)(x)
    x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
    x = Activation('relu', name='dense_act_%i' % i)(x)

x = Dense(int(n_classes), name='output_dense')(x)
x_out = Activation('softmax', name='output_softmax')(x)

keras_model = Model(inputs=[x_in], outputs=[x_out], name='alexnet')

keras_model.summary()


Adding convolutional block 0 with N=64 filters


2023-06-28 15:19:22.420126: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 15:19:22.421286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 15:19:22.421477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-28 15:19:22.421535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

# Train

In [6]:
datagen = ImageDataGenerator(
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def step_decay(epoch):
    initial_lr = lr
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * drop * (epoch // epochs_drop)
    return lr

#train_datagen = datagen.flow(train_images, train_labels, batch_size=batch_size)

keras_model.compile(
    optimizer=Adam(lr),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir_keras,
    histogram_freq=1
)
start = time.time()
keras_model.fit(
    #datagen.flow(train_images, train_labels, batch_size=batch_size),
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(test_images, test_labels),
    verbose=1,
    callbacks=[tensorboard_callback]
)
end = time.time()

keras_model.save(model_name)
print('It took {} minutes to train Keras model'.format((end - start) / 60.0))

2023-06-12 04:19:05.937835: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-12 04:19:05.937853: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-12 04:19:05.937880: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2023-06-12 04:19:05.938056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-12 04:19:05.938122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-12 04:19:05.938137: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1

Epoch 1/200


2023-06-12 04:19:07.877935: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-12 04:19:08.329596: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-06-12 04:19:08.892071: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-12 04:19:09.301795: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


15/98 [===>..........................] - ETA: 1s - loss: 3.3074 - accuracy: 0.1443

2023-06-12 04:19:09.718167: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-12 04:19:09.718188: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-12 04:19:09.718213: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-12 04:19:09.732027: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-06-12 04:19:09.732068: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-12 04:19:09.734548: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-06-12 04:19:09.735573: I tensorflow

98/98 [==============================] - 5s 20ms/step - loss: 2.9645 - accuracy: 0.2639 - val_loss: 3.3528 - val_accuracy: 0.0855
Epoch 2/200
98/98 [==============================] - 1s 13ms/step - loss: 2.4982 - accuracy: 0.3814 - val_loss: 3.6215 - val_accuracy: 0.1084
Epoch 3/200
98/98 [==============================] - 1s 13ms/step - loss: 2.1754 - accuracy: 0.4522 - val_loss: 3.7005 - val_accuracy: 0.1062
Epoch 4/200
98/98 [==============================] - 1s 13ms/step - loss: 1.9560 - accuracy: 0.4983 - val_loss: 3.0332 - val_accuracy: 0.1614
Epoch 5/200
98/98 [==============================] - 1s 13ms/step - loss: 1.8000 - accuracy: 0.5374 - val_loss: 2.3296 - val_accuracy: 0.3418
Epoch 6/200
98/98 [==============================] - 1s 14ms/step - loss: 1.6927 - accuracy: 0.5613 - val_loss: 2.0990 - val_accuracy: 0.4090
Epoch 7/200
98/98 [==============================] - 1s 13ms/step - loss: 1.6053 - accuracy: 0.5850 - val_loss: 1.8838 - val_accuracy: 0.4804
Epoch 8/200
98/98 

98/98 [==============================] - 1s 13ms/step - loss: 0.9286 - accuracy: 0.7570 - val_loss: 1.3598 - val_accuracy: 0.6381
Epoch 116/200
98/98 [==============================] - 1s 13ms/step - loss: 0.9166 - accuracy: 0.7628 - val_loss: 1.2711 - val_accuracy: 0.6578
Epoch 117/200
98/98 [==============================] - 1s 13ms/step - loss: 0.9218 - accuracy: 0.7600 - val_loss: 1.3784 - val_accuracy: 0.6300
Epoch 118/200
98/98 [==============================] - 1s 14ms/step - loss: 0.9128 - accuracy: 0.7648 - val_loss: 1.1908 - val_accuracy: 0.6764
Epoch 119/200
98/98 [==============================] - 1s 13ms/step - loss: 0.9148 - accuracy: 0.7637 - val_loss: 1.2932 - val_accuracy: 0.6556
Epoch 120/200
98/98 [==============================] - 1s 13ms/step - loss: 0.9201 - accuracy: 0.7614 - val_loss: 1.2126 - val_accuracy: 0.6735
Epoch 121/200
98/98 [==============================] - 1s 13ms/step - loss: 0.9135 - accuracy: 0.7628 - val_loss: 1.2384 - val_accuracy: 0.6670
Epoch 

Epoch 172/200
98/98 [==============================] - 1s 14ms/step - loss: 0.8945 - accuracy: 0.7693 - val_loss: 1.2384 - val_accuracy: 0.6652
Epoch 173/200
98/98 [==============================] - 1s 13ms/step - loss: 0.8909 - accuracy: 0.7694 - val_loss: 1.3892 - val_accuracy: 0.6269
Epoch 174/200
98/98 [==============================] - 2s 18ms/step - loss: 0.8962 - accuracy: 0.7649 - val_loss: 1.5821 - val_accuracy: 0.5732
Epoch 175/200
98/98 [==============================] - 1s 13ms/step - loss: 0.8902 - accuracy: 0.7694 - val_loss: 1.7445 - val_accuracy: 0.5479
Epoch 176/200
98/98 [==============================] - 1s 14ms/step - loss: 0.8907 - accuracy: 0.7700 - val_loss: 1.1455 - val_accuracy: 0.6913
Epoch 177/200
98/98 [==============================] - 1s 14ms/step - loss: 0.8880 - accuracy: 0.7698 - val_loss: 1.2137 - val_accuracy: 0.6833
Epoch 178/200
98/98 [==============================] - 1s 14ms/step - loss: 0.8901 - accuracy: 0.7702 - val_loss: 1.2637 - val_accuracy: